# Visualise Starfield Resource Value:Mass Ratio

In [1]:
from pyprojroot import here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ridgeplot import ridgeplot
from itables import (init_notebook_mode, show)

init_notebook_mode(all_interactive=False)


<IPython.core.display.Javascript object>

In [2]:
df = pd.read_pickle("starfield-resources.pkl")
df.reset_index(names="resource", inplace=True)
df["resource"] = pd.Categorical(df["resource"])

In [3]:
starfield_pal = {
    "dark-navy": "#1c1e1f",
    "navy": "#213150",
    "dark-blue": "#2f4c79",
    "light-blue": "#3b557a",
    "yellow": "#d8ab61",
    "orange": "#df6035",
    "red": "#c72336",
    "grey": "#57595b",
    "white": "#f4f6f8",
}

In [5]:
cmap = { # replaces default color mapping by value
                "Common": starfield_pal["red"],
                "Uncommon": starfield_pal["orange"],
                "Rare": starfield_pal["yellow"],
                "Exotic": starfield_pal["light-blue"],
                "Unique": starfield_pal["dark-blue"]
            }

fig = px.scatter(
    df, x="mass", y="value", color="rarity",
    hover_data="resource",
    color_discrete_map=cmap,
    trendline="ols",
    trendline_scope="overall",
    trendline_color_override=starfield_pal["white"],
    # control order of legend
    category_orders={"rarity": ["Common", "Uncommon", "Rare", "Exotic", "Unique"]},
    title="Starfield Resources: Comparing Mass & Value by Rarity",
    )
fig.update_layout(
        plot_bgcolor=starfield_pal["dark-navy"],
        paper_bgcolor=starfield_pal["dark-navy"],
        legend_font_color=starfield_pal["white"],
        font_color=starfield_pal["white"],
        annotations = [dict(
        x=0.008,
        y=1.05,
        xref="paper",
        yref="paper",
        text="Source: https://inara.cz/starfield/resources/",
        showarrow = False
    )]
        )
fig.update_xaxes(
            gridcolor=starfield_pal["grey"], zeroline=False,
            )
fig.update_yaxes(gridcolor=starfield_pal["grey"], zeroline=False)
fig.update_traces(marker=dict(size=9))
                            
fig.show()

In [6]:
# save output
fig.write_html("outputs/starfield-resources-scatter.html")

In [7]:
rat_rar = df[["ratio", "rarity"]]

In [8]:
rat_rar = rat_rar.pivot(columns="rarity", values="ratio")
rat_rar = rat_rar.fillna(value=0.0)
rat_rar = rat_rar[["Common", "Uncommon", "Rare", "Exotic", "Unique"]]

In [9]:
lim = 200
fig = ridgeplot(
    samples=rat_rar[rat_rar.le(lim).all(axis=1)].values.T, labels=rat_rar.columns,
    linewidth=2,
    colorscale="portland",
    colormode="row-index", coloralpha=0.65,
    bandwidth=5,# standardises height of trace,
    # colorscale=cmap,

                )
fig.update_layout(
    font_size=12,
    plot_bgcolor=starfield_pal["dark-navy"],
    xaxis_gridcolor=starfield_pal["dark-navy"],
    yaxis_gridcolor=starfield_pal["grey"],
    showlegend=False,
    xaxis_title="Value to Mass Ratio (credits per g)",
    yaxis_title="Density (kde)",
    title="Density Plot of Resource Value:Mass Ratio by Rarity in Starfield",
    height=760,
    width=900,
    paper_bgcolor=starfield_pal["dark-navy"],
    font_color=starfield_pal["white"],
    annotations = [dict(
        x=0,
        y=1.05,
        xref="paper",
        yref="paper",
        text=f"Ratio filtered to less than {lim} credits per gram",
        showarrow = False
    )]


)
fig.show()

In [13]:
# for completeness, the values filtered out are:
df.query("ratio > @lim")

,resource,mass,value,ratio,rarity
0,Caelumite (Ct),0.3,264,880.0,Unique
1,Quark-Degenerate Tissues,0.3,240,800.0,Unique


In [10]:
fig.write_html("outputs/starfield-resources-joyplot.html")

In [16]:
show(df)
